# Tutorial

This is the SQLAlchemy Tutorial found in: http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

## Connecting

In [1]:
import sqlalchemy as sql

To connect to a Database, use the `create_engine` function

In [2]:
engine = sql.create_engine('sqlite:///:memory:', echo=True)

## Declaring a Mapping

The `declarative` system allows to create classes that include direcetives to describe the actual database table they will be mapped to - DB descrption + Class definition

In [3]:
from sqlalchemy.ext.declarative import declarative_base

In [4]:
base = declarative_base()

If the class is using base, it requires at a minimum:
* `__tablename__`
* One Column with primary key

In [5]:
from sqlalchemy import Column, Integer, String
class User(base):
    __tablename__ = 'users'
    
    id = Column(Integer,primary_key=True)
    name = Column(String)
    #name = Column(String(50)) --> To provide the length when VARCHAR requires it
    fullname = Column(String)
    password = Column(String)
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

## Schema Creation

In [6]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [7]:
base.metadata.create_all(engine)

2017-06-29 11:08:08,589 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-06-29 11:08:08,605 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 11:08:08,607 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-06-29 11:08:08,608 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 11:08:08,614 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-06-29 11:08:08,615 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 11:08:08,619 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-06-29 11:08:08,621 INFO sqlalchemy.engine.base.Engine ()
2017-06-29 11:08:08,625 INFO sqlalchemy.engine.base.Engine COMMIT


## Creating an Instance

In [8]:
end_user = User(name = 'Juan',fullname = 'Juan Daza', password = 'password')

In [9]:
end_user.name

'Juan'

In [10]:
str(end_user.id)

'None'

## Creating a Session

In [11]:
from sqlalchemy.orm import sessionmaker

In [12]:
Session = sessionmaker(bind = engine)

In [13]:
session = Session()

In [14]:
session.add(end_user)

In [15]:
user = session.query(User).filter_by(name = 'Juan').first()

2017-06-29 11:08:10,358 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-06-29 11:08:10,361 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 11:08:10,362 INFO sqlalchemy.engine.base.Engine ('Juan', 'Juan Daza', 'password')
2017-06-29 11:08:10,368 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-06-29 11:08:10,369 INFO sqlalchemy.engine.base.Engine ('Juan', 1, 0)


In [16]:
print user

<User(name='Juan', fullname='Juan Daza', password='password')>


In [17]:
user == end_user

True

In [18]:
session.add_all([User(name='John', fullname='John Doe', password='password'),
                User(name='Charles', fullname='Charles Xavier', password='password'),
                User(name='Logan', fullname='Logan Xavier', password='password')])

In [19]:
user.password = 'change1'

In [20]:
user = session.query(User).filter_by(name = 'Juan').first()

2017-06-29 11:08:11,864 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-06-29 11:08:11,865 INFO sqlalchemy.engine.base.Engine ('change1', 1)
2017-06-29 11:08:11,868 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 11:08:11,871 INFO sqlalchemy.engine.base.Engine ('John', 'John Doe', 'password')
2017-06-29 11:08:11,872 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 11:08:11,875 INFO sqlalchemy.engine.base.Engine ('Charles', 'Charles Xavier', 'password')
2017-06-29 11:08:11,878 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 11:08:11,880 INFO sqlalchemy.engine.base.Engine ('Logan', 'Logan Xavier', 'password')
2017-06-29 11:08:11,884 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS u

In [21]:
print user

<User(name='Juan', fullname='Juan Daza', password='change1')>


In [22]:
end_user.password = 'change2'

In [23]:
user = session.query(User).filter_by(name = 'Juan').first()

2017-06-29 11:08:14,334 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-06-29 11:08:14,335 INFO sqlalchemy.engine.base.Engine ('change2', 1)
2017-06-29 11:08:14,339 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-06-29 11:08:14,342 INFO sqlalchemy.engine.base.Engine ('Juan', 1, 0)


In [24]:
print user

<User(name='Juan', fullname='Juan Daza', password='change2')>


In [25]:
session.dirty

IdentitySet([])

In [26]:
session.new

IdentitySet([])

In [27]:
session.commit()

2017-06-29 11:08:15,684 INFO sqlalchemy.engine.base.Engine COMMIT


In [28]:
user.id

2017-06-29 11:08:15,821 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-06-29 11:08:15,823 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2017-06-29 11:08:15,825 INFO sqlalchemy.engine.base.Engine (1,)


1

## Rolling Back

In [29]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

In [30]:
session.commit()

2017-06-29 11:08:16,244 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 11:08:16,246 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2017-06-29 11:08:16,250 INFO sqlalchemy.engine.base.Engine COMMIT


The following rollback will not work as the commit command will already be executed - The DB requires now that the user is deleted

In [31]:
session.rollback()

In [32]:
fake_user in session

True

In [33]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2017-06-29 11:08:17,670 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-06-29 11:08:17,673 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2017-06-29 11:08:17,674 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='fakeuser', fullname='Invalid', password='12345')>]

In [34]:
rollback_user = User(name='fakeuser', fullname='Invalid', password='12345')

In [35]:
session.add(rollback_user)

In [36]:
session.query(User).filter(User.name.in_(['fakeuser'])).all()

2017-06-29 11:08:18,101 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-06-29 11:08:18,102 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2017-06-29 11:08:18,107 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?)
2017-06-29 11:08:18,108 INFO sqlalchemy.engine.base.Engine ('fakeuser',)


[<User(name='fakeuser', fullname='Invalid', password='12345')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [37]:
session.rollback()

2017-06-29 11:08:19,020 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [38]:
rollback_user in session

False